In [2]:
import os
os.chdir(r"/Users/rakshaypawar/Desktop/Problem Set 6_2022")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Nelson-Siegel-Svensson Function for ZCB yields
#note that the early years ignore t2 and b3, but no need to adjust the formula since b3=0
def nssyld(b0, b1, b2, b3, t1, t2, m):
    mt1 = m/t1
    mt2 = m/t2    
    expmt1 = np.exp(-mt1)
    expmt2 = np.exp(-mt2)
    c1 = (1-expmt1)/mt1
    c2 = c1 - expmt1
    c3 = (1-expmt2)/mt2 - expmt2
    return(np.exp((b0 + b1 * c1 + b2 * c2 + b3 * c3)/100)-1)  #total the returns

begindate = "1971-12-31"
enddate = "2021-12-31"

#read the data, beginning with the S&P
#sp500
sp500 = pd.read_csv("sp500_2022.csv")
sp500.columns = ['date','sp500']
sp500['date'] = pd.to_datetime(sp500['date'], format = '%Y%m%d')

filt = (sp500['date']>= begindate) & (sp500['date'] <= enddate)
sp500 = sp500.loc[filt]

sp500.loc[-1] = [pd.to_datetime("1971-12-31"),0]
sp500.index = sp500.index + 1  # shifting index
sp500.sort_index(inplace=True) 

#work on the yield curve
#The following assumes headers have already been removed and the first column is named DATE
nss = pd.read_csv("nss_2022.csv")
nss.columns = nss.columns.str.lower()
nss['date'] = pd.to_datetime(nss['date'], format = '%m/%d/%Y')
nss = nss.dropna( how='any', subset=["beta0", "beta1", "beta2", "beta3", "tau1", "tau2"])
filt = (nss['date']>= begindate) & (nss['date'] <= enddate)
nss = nss.loc[filt]

#merge the datasets 
d = sp500.merge(nss, on='date', how='left')

# The bond market is open fewer days than the stock market.
d = d.dropna()

d['sp500_plus_1'] = d['sp500'] + 1
d['sp500cumulative'] = d['sp500_plus_1'].cumprod(axis = 0)
d = d.dropna()

d.set_index('date', inplace=True)
d['date'] = d.index

#consolidate returns by time horizon
def sp500rhorizon(d, horizon):
    n = d.resample(horizon).last()
    return n

def findyears(d):
    x = ((max(d['date']) - min(d['date'])).days)/365.25
    x= round(x)
    return x


#build the returns at each horizon
m = sp500rhorizon(d, horizon="M")
y = sp500rhorizon(m, horizon="Y")
y5 = y.iloc[::5, :] #y.iloc[::5,:] > extract every 5th row 

m['r'] = m['sp500cumulative']/m['sp500cumulative'].shift() - 1
y['r'] = y['sp500cumulative']/y['sp500cumulative'].shift() - 1
y5['r'] = y5['sp500cumulative']/y5['sp500cumulative'].shift() - 1
d['r']=d['sp500']

#adds in the ZCB info and computes the excess returns
def excessrets(d):
    d['mat'] = (d['date'].shift(-1) - d['date']).dt.days/365  # maturities of ZCB (ACT/365)
    d['zcb'] = d.apply(lambda row: nssyld(row['beta0'], row['beta1'], row['beta2'], row['beta3'], row['tau1'],row['tau2'],row['mat']),axis =1) # ZCB yields
    # ZCB ret (no need to go through the PS4 approach 
    # since these are matched with the time horizon)
    d['rf'] = d['zcb'].shift()*d['mat'].shift()
    d['xret'] = d['r'] - d['rf']
    filt = (d['date'] !=  "1971-12-31")
    d = d.loc[filt] #drop first row 
    d['xret_1'] = d['xret'] +1
    d['xlevel'] = d['xret_1'].cumprod(axis = 0)
    return d

nyears = {'d': findyears(d),
          'm': findyears(m),
          'y': findyears(y),
          'y5': (len(y.iloc[::5,:])-1)*5}

d = excessrets(d)
m = excessrets(m)
y = excessrets(y)
y5 = excessrets(y5)

testd = d
testm = m
testy = y
testy5 = y5

# enter the results 
days_per_year = d.shape[0]/ nyears['d']
res = pd.DataFrame(np.zeros((4,4)), columns =['Arith SP500', 'Geom SP500', 'Arith Excess', 'Geom Excess'])
res.index = ["Daily Returns", "Monthly Returns", "Annual Returns", "5-year Returns"]
res['Arith SP500'].iloc[0] = d['r'].mean()*days_per_year
res['Arith SP500'].iloc[1] = m['r'].mean()*12
res['Arith SP500'].iloc[2] = y['r'].mean()
res['Arith SP500'].iloc[3] = y5['r'].mean()/5

res['Geom SP500'].iloc[0] =  d['sp500cumulative'].iloc[-1] **(1 / nyears['d']) - 1
res['Geom SP500'].iloc[1] =  m['sp500cumulative'].iloc[-1] **(1 / nyears['m']) - 1
res['Geom SP500'].iloc[2] =  y['sp500cumulative'].iloc[-1] **(1 / nyears['y']) - 1
res['Geom SP500'].iloc[3] = y5['sp500cumulative'].iloc[-1] **(1 / nyears['y5']) - 1

res['Arith Excess'].iloc[0] = d['xret'].mean()*days_per_year
res['Arith Excess'].iloc[1] = m['xret'].mean()*12
res['Arith Excess'].iloc[2] = y['xret'].mean()
res['Arith Excess'].iloc[3] = y5['xret'].mean()/5

res['Geom Excess'].iloc[0] = d['xlevel'].iloc[-1]**(1 / nyears['d']) -1 
res['Geom Excess'].iloc[1] = m['xlevel'].iloc[-1]**(1 / nyears['m']) -1 
res['Geom Excess'].iloc[2] = y['xlevel'].iloc[-1]**(1 / nyears['y']) -1 
res['Geom Excess'].iloc[3] = y5['xlevel'].iloc[-1]**(1 / nyears['y5']) -1 


res = res*100
print(res)

                 Arith SP500  Geom SP500  Arith Excess  Geom Excess
Daily Returns      11.102462   10.115897      6.365313     5.020206
Monthly Returns    10.848059   10.115897      6.069430     4.990674
Annual Returns     11.597021   10.115897      6.530359     4.925856
5-year Returns     13.793386   10.115897      7.937892     5.582103


In [3]:
res3 = pd.DataFrame(np.zeros((4,2)), columns =["w: SP500", "w: Bonds"])
res3.index = ["Daily Returns", "Monthly Returns", "Annual Returns", "5-year Returns"]
A = 4

def wsp500(xret):
    result = xret.mean()/ (A*xret.var())
    return result

In [4]:
res3["w: SP500"].iloc[0] = wsp500(d['xret'])
res3["w: SP500"].iloc[1] = wsp500(m['xret'])
res3["w: SP500"].iloc[2] = wsp500(y['xret'])
res3["w: SP500"].iloc[3] = wsp500(y5['xret'])
res3["w: Bonds"]= 1 - res3["w: SP500"]
res3_100=res3*100

In [5]:
print(res3_100)

                  w: SP500   w: Bonds
Daily Returns    54.577936  45.422064
Monthly Returns  65.127854  34.872146
Annual Returns   54.186172  45.813828
5-year Returns   40.158581  59.841419


In [6]:
print(res.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Arith SP500 &  Geom SP500 &  Arith Excess &  Geom Excess \\
\midrule
Daily Returns   &    11.102462 &   10.115897 &      6.365313 &     5.020206 \\
Monthly Returns &    10.848059 &   10.115897 &      6.069430 &     4.990674 \\
Annual Returns  &    11.597021 &   10.115897 &      6.530359 &     4.925856 \\
5-year Returns  &    13.793386 &   10.115897 &      7.937892 &     5.582103 \\
\bottomrule
\end{tabular}



In [7]:
print(res3_100.to_latex())

\begin{tabular}{lrr}
\toprule
{} &   w: SP500 &   w: Bonds \\
\midrule
Daily Returns   &  54.577936 &  45.422064 \\
Monthly Returns &  65.127854 &  34.872146 \\
Annual Returns  &  54.186172 &  45.813828 \\
5-year Returns  &  40.158581 &  59.841419 \\
\bottomrule
\end{tabular}

